# Ecological Index - table information
- Print statistics **related to total area**
- Notebook by Alice DuVivier (NCAR)
- Note: this notebook needs to load the utils.py file
- November 2024

In [1]:
import utils
import xarray as xr
import numpy as np
from glob import glob
import intake
import importlib
import pop_tools
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import matplotlib.ticker as mticker
import cartopy
import cartopy.crs as ccrs
import cmocean
import geopandas as gpd

### Load grid and mask data

In [2]:
# directory path
dir_in = '/glade/u/home/duvivier/masks/'

In [3]:
# load ocean grid
fin = 'ocn_grid_gx1v7.nc'
ds_grid = xr.open_mfdataset(dir_in+fin,decode_times=False)
ds_grid['nlat'] = ds_grid.nlat
ds_grid['nlon'] = ds_grid.nlon
ds_grid

<xarray.Dataset>
Dimensions:                 (moc_comp: 3, transport_comp: 5, transport_reg: 2, z_t: 60, z_t_150m: 15, z_w: 60, z_w_top: 60, z_w_bot: 60, lat_aux_grid: 395, moc_z: 61, nlat: 384, nlon: 320, time: 1188, d2: 2)
Coordinates: (12/14)
  * z_t                     (z_t) float32 500.0 1.5e+03 ... 5.125e+05 5.375e+05
  * z_t_150m                (z_t_150m) float32 500.0 1.5e+03 ... 1.45e+04
  * z_w                     (z_w) float32 0.0 1e+03 2e+03 ... 5e+05 5.25e+05
  * z_w_top                 (z_w_top) float32 0.0 1e+03 2e+03 ... 5e+05 5.25e+05
  * z_w_bot                 (z_w_bot) float32 1e+03 2e+03 ... 5.25e+05 5.5e+05
  * lat_aux_grid            (lat_aux_grid) float32 -79.49 -78.95 ... 89.47 90.0
    ...                      ...
    ULAT                    (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    TLONG                   (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    TLAT                    (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
  * time                    (time) float32 31.0 59.0 90.0 ... 3.61e+04 3.614e+04
  * nlat                    (nlat) int64 0 1 2 3 4 5 ... 378 379 380 381 382 383
  * nlon                    (nlon) int64 0 1 2 3 4 5 ... 314 315 316 317 318 319
Dimensions without coordinates: moc_comp, transport_comp, transport_reg, d2
Data variables: (12/58)
    moc_components          (moc_comp) |S384 dask.array<chunksize=(3,), meta=np.ndarray>
    transport_components    (transport_comp) |S384 dask.array<chunksize=(5,), meta=np.ndarray>
    transport_regions       (transport_reg) |S384 dask.array<chunksize=(2,), meta=np.ndarray>
    dz                      (z_t) float32 dask.array<chunksize=(60,), meta=np.ndarray>
    dzw                     (z_w) float32 dask.array<chunksize=(60,), meta=np.ndarray>
    KMT                     (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    ...                      ...
    nsurface_u              float64 ...
    time_bound              (time, d2) float64 dask.array<chunksize=(1188, 2), meta=np.ndarray>
    QFLUX                   (time, nlat, nlon) float32 dask.array<chunksize=(1188, 384, 320), meta=np.ndarray>
    SHF                     (time, nlat, nlon) float32 dask.array<chunksize=(1188, 384, 320), meta=np.ndarray>
    SHF_QSW                 (time, nlat, nlon) float32 dask.array<chunksize=(1188, 384, 320), meta=np.ndarray>
    TEMP                    (time, z_t, nlat, nlon) float32 dask.array<chunksize=(1188, 60, 384, 320), meta=np.ndarray>
Attributes:
    title:             b.e21.B1850.f09_g17.CMIP6-piControl.001
    history:           none
    Conventions:       CF-1.0; http://www.cgd.ucar.edu/cms/eaton/netcdf/CF-cu...
    time_period_freq:  month_1
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    contents:          Diagnostic and Prognostic Variables
    source:            CCSM POP2, the CCSM Ocean Component
    revision:          $Id: tavg.F90 89644 2018-08-04 14:26:01Z klindsay $
    calendar:          All years have exactly  365 days.
    start_time:        This dataset was created on 2018-08-09 at 18:18:26.3
    cell_methods:      cell_methods = time: mean ==> the variable values are ...

In [4]:
## Enter the latitudes you want to keep:
lat_min = -80
lat_max = -60

In [5]:
# crop data to the latitudes we want, use given lat/lon, not specific indices
ds_grid = ds_grid.where(((ds_grid['TLAT'] <= lat_max) & (ds_grid['TLAT'] >= lat_min)), drop=True)

In [6]:
# save tarea and convert to km2
cm2_to_km2 = 100.0 * 100.0 * 1000.0 * 1000.0
tarea = ds_grid.TAREA/cm2_to_km2

## Load EcoIndex Data

In [7]:
path_in = '/glade/campaign/cgd/ppc/duvivier/cesm2_antarctic_polynya/mpa_analysis/DATA/ecoindex_data/ecoindex/'

### Load "Present Day"

In [8]:
file_in = 'ecoindex_present_day.nc'
ds_eco = xr.open_dataset(path_in+file_in, decode_times=True)

# mask land
ds_eco = ds_eco.where(ds_grid.REGION_MASK == 1)

In [9]:
# keep regional version only
eco_pd = ds_eco.ecoindex_regional_noseal

### Load model data

In [10]:
file_in = 'ecoindex_historical_future.nc'
ds_eco = xr.open_dataset(path_in+file_in, decode_times=True)

# mask land
ds_eco = ds_eco.where(ds_grid.REGION_MASK == 1)

In [11]:
# keep some decades
eco_2000 = ds_eco.ecoindex_regional_decadal.sel(decades=2000)

In [12]:
# keep some decades
eco_2090 = ds_eco.ecoindex_regional_decadal.sel(decades=2090)

### Make masks of "very high" and "high" region

- Very high: EI>= 0.8
- High: EI>=0.6 AND EI<0.8
- Moderate: EI>=0.4 AND EI<0.6
- Low: EI<0.4

In [13]:
#eco_bin_names = ['very_high','high','moderate','low']
eco_bin_names = [1,2,3,4]

In [14]:
# combine these into a single array
xlen = len(eco_pd.nlon)
ylen = len(eco_pd.nlat)

In [15]:
# "Present Day" masking
data = eco_pd

# make arrays to fill
eco_1 = data.where(data == 1, other =1)
eco_2 = data.where(data == 1, other =1)
eco_3 = data.where(data == 1, other =1)
eco_4 = data.where(data == 1, other =1)

# now just mask by bin
eco_1 = eco_1.where(data >=0.8)
eco_2 = eco_2.where((data >=0.6) & (data <0.8))
eco_3 = eco_3.where((data >=0.4) & (data <0.6))
eco_4 = eco_4.where(data <0.4)

# convert to single array
eco_bins = np.zeros([4,ylen,xlen])
eco_bins[0,:,:] = eco_1.values
eco_bins[1,:,:] = eco_2.values
eco_bins[2,:,:] = eco_3.values
eco_bins[3,:,:] = eco_4.values

# convert to xarray
eco_bins_pd = xr.DataArray(eco_bins,dims=('bins','nlat','nlon'))
eco_bins_pd['bins'] = eco_bin_names
eco_bins_pd['nlat'] = data.nlat
eco_bins_pd['nlon'] = data.nlon

In [16]:
# 2000 masking
data = eco_2000

# make arrays to fill
eco_1 = data.where(data == 1, other =1)
eco_2 = data.where(data == 1, other =1)
eco_3 = data.where(data == 1, other =1)
eco_4 = data.where(data == 1, other =1)

# now just mask by bin
eco_1 = eco_1.where(data >=0.8)
eco_2 = eco_2.where((data >=0.6) & (data <0.8))
eco_3 = eco_3.where((data >=0.4) & (data <0.6))
eco_4 = eco_4.where(data <0.4)

# convert to single array
eco_bins = np.zeros([4,ylen,xlen])
eco_bins[0,:,:] = eco_1.values
eco_bins[1,:,:] = eco_2.values
eco_bins[2,:,:] = eco_3.values
eco_bins[3,:,:] = eco_4.values

# convert to xarray
eco_bins_2000 = xr.DataArray(eco_bins,dims=('bins','nlat','nlon'))
eco_bins_2000['bins'] = eco_bin_names
eco_bins_2000['nlat'] = data.nlat
eco_bins_2000['nlon'] = data.nlon

In [17]:
# 2090 masking
data = eco_2090

# make arrays to fill
eco_1 = data.where(data == 1, other =1)
eco_2 = data.where(data == 1, other =1)
eco_3 = data.where(data == 1, other =1)
eco_4 = data.where(data == 1, other =1)

# now just mask by bin
eco_1 = eco_1.where(data >=0.8)
eco_2 = eco_2.where((data >=0.6) & (data <0.8))
eco_3 = eco_3.where((data >=0.4) & (data <0.6))
eco_4 = eco_4.where(data <0.4)

# convert to single array
eco_bins = np.zeros([4,ylen,xlen])
eco_bins[0,:,:] = eco_1.values
eco_bins[1,:,:] = eco_2.values
eco_bins[2,:,:] = eco_3.values
eco_bins[3,:,:] = eco_4.values

# convert to xarray
eco_bins_2090 = xr.DataArray(eco_bins,dims=('bins','nlat','nlon'))
eco_bins_2090['bins'] = eco_bin_names
eco_bins_2090['nlat'] = data.nlat
eco_bins_2090['nlon'] = data.nlon

## Load map related masks for analysis

In [18]:
dir_in = '/glade/campaign/cgd/ppc/duvivier/cesm2_antarctic_polynya/mpa_analysis/DATA/'

In [19]:
# Read in CCAMLR planning domain masks
file_in = 'CCAMLR_masks/CESM_gx1v7_CCAMLR_masks_v8_10102024.nc'
ds_ccamlr= xr.open_mfdataset(dir_in+file_in,decode_times=False)
ds_ccamlr = ds_ccamlr.isel(nlat=slice(0,36))

# keep just masks of interest (MPAs, MPA planning areas, AES)
ccamlr_in = ['mpa_plan_1_mod','mpa_plan_3_4','mpa_plan_7_mod','mpa_plan_8','mpa_plan_9',
             'Ross_mpa','Weddell_mpa','Ant_Peninsula_mpa','East_Ant_mpa',
             'AES']
ccamlr_masks = ds_ccamlr[ccamlr_in]

# now load in the different masks as a new coordinate
ccamlr_masks = xr.concat([ds_ccamlr.mpa_plan_1_mod, ds_ccamlr.mpa_plan_3_4, ds_ccamlr.mpa_plan_7_mod,
                          ds_ccamlr.mpa_plan_8, ds_ccamlr.mpa_plan_9,
                          ds_ccamlr.Ross_mpa, ds_ccamlr.Weddell_mpa,
                          ds_ccamlr.Ant_Peninsula_mpa, ds_ccamlr.East_Ant_mpa,
                          ds_ccamlr.AES],
                         dim='region')
ccamlr_masks['region']=ccamlr_in


# Calculate statistics for Total Area

## Hemispheric
- Calculate total fractions High and Exceptional

In [22]:
# keep only tarea where we have ecoindex data (no land points)
tarea_sub = tarea.where(ds_grid.REGION_MASK == 1)

In [23]:
# multiply binned data by tarea
tarea_pd_bins = eco_bins_pd * tarea
tarea_2000_bins = eco_bins_2000 * tarea
tarea_2090_bins = eco_bins_2090 * tarea

#### Print stats by bin

In [24]:
print('Exceptional area fractions')
bin = 1

# calculate and print hemispheric fraction of area that is this bin
print('Observational percent bin hemispherically')
frac_pd = tarea_pd_bins.sel(bins=bin).sum(dim=['nlat','nlon'])/tarea_sub.sum(dim=['nlat','nlon'])
print(frac_pd.values * 100.0)

print('Model 2000s percent bin hemispherically')
frac_2000 = tarea_2000_bins.sel(bins=bin).sum(dim=['nlat','nlon'])/tarea_sub.sum(dim=['nlat','nlon'])
print(frac_2000.values * 100.0)

print('Model 2090s percent bin hemispherically')
frac_2090 = tarea_2090_bins.sel(bins=bin).sum(dim=['nlat','nlon'])/tarea_sub.sum(dim=['nlat','nlon'])
print(frac_2090.values * 100.0)

Exceptional area fractions
Observational percent bin hemispherically
1.273439218204328
Model 2000s percent bin hemispherically
1.5352239868379047
Model 2090s percent bin hemispherically
1.5621049529614053


In [25]:
print('High area fractions')
bin = 2

# calculate and print hemispheric fraction of area that is this bin
print('Observational percent bin hemispherically')
frac_pd = tarea_pd_bins.sel(bins=bin).sum(dim=['nlat','nlon'])/tarea_sub.sum(dim=['nlat','nlon'])
print(frac_pd.values * 100.0)

print('Model 2000s percent bin hemispherically')
frac_2000 = tarea_2000_bins.sel(bins=bin).sum(dim=['nlat','nlon'])/tarea_sub.sum(dim=['nlat','nlon'])
print(frac_2000.values * 100.0)

print('Model 2090s percent bin hemispherically')
frac_2090 = tarea_2090_bins.sel(bins=bin).sum(dim=['nlat','nlon'])/tarea_sub.sum(dim=['nlat','nlon'])
print(frac_2090.values * 100.0)

High area fractions
Observational percent bin hemispherically
3.157364735629976
Model 2000s percent bin hemispherically
2.8677070129384385
Model 2090s percent bin hemispherically
3.5479140642266627


In [26]:
print('Moderate area fractions')
bin = 3

# calculate and print hemispheric fraction of area that is this bin
print('Observational percent bin hemispherically')
frac_pd = tarea_pd_bins.sel(bins=bin).sum(dim=['nlat','nlon'])/tarea_sub.sum(dim=['nlat','nlon'])
print(frac_pd.values * 100.0)

print('Model 2000s percent bin hemispherically')
frac_2000 = tarea_2000_bins.sel(bins=bin).sum(dim=['nlat','nlon'])/tarea_sub.sum(dim=['nlat','nlon'])
print(frac_2000.values * 100.0)

print('Model 2090s percent bin hemispherically')
frac_2090 = tarea_2090_bins.sel(bins=bin).sum(dim=['nlat','nlon'])/tarea_sub.sum(dim=['nlat','nlon'])
print(frac_2090.values * 100.0)

Moderate area fractions
Observational percent bin hemispherically
24.717758802662775
Model 2000s percent bin hemispherically
24.330277797871652
Model 2090s percent bin hemispherically
24.719968793101437


In [27]:
print('Low area fractions')
bin = 4

# calculate and print hemispheric fraction of area that is this bin
print('Observational percent bin hemispherically')
frac_pd = tarea_pd_bins.sel(bins=bin).sum(dim=['nlat','nlon'])/tarea_sub.sum(dim=['nlat','nlon'])
print(frac_pd.values * 100.0)

print('Model 2000s percent bin hemispherically')
frac_2000 = tarea_2000_bins.sel(bins=bin).sum(dim=['nlat','nlon'])/tarea_sub.sum(dim=['nlat','nlon'])
print(frac_2000.values * 100.0)

print('Model 2090s percent bin hemispherically')
frac_2090 = tarea_2090_bins.sel(bins=bin).sum(dim=['nlat','nlon'])/tarea_sub.sum(dim=['nlat','nlon'])
print(frac_2090.values * 100.0)

Low area fractions
Observational percent bin hemispherically
70.8514372435029
Model 2000s percent bin hemispherically
71.26679120235198
Model 2090s percent bin hemispherically
70.17001218971048


## Regional
- Calculate area weighted average AEV for polynyas regionally
    - ds_ccamlr.mpa_plan_1_mod
    - ds_ccamlr.mpa_plan_3_4
    - ds_ccamlr.mpa_plan_7_mod
    - ds_ccamlr.mpa_plan_8
    - ds_ccamlr.mpa_plan_9

In [28]:
# keep only tarea where we have ecoindex data (no land points) and by region
tarea_region = tarea_sub * ccamlr_masks

In [29]:
# multiply binned data by tarea and by region
tarea_pd_bins = eco_bins_pd * tarea * ccamlr_masks
tarea_2000_bins = eco_bins_2000 * tarea * ccamlr_masks
tarea_2090_bins = eco_bins_2090 * tarea * ccamlr_masks

#### Exceptional

In [30]:
print('Fraction of total area by region that is EXCEPTIONAL AEV value')
# keep exceptional points
bin = 1

tarea_sub_pd = tarea_pd_bins.sel(bins=bin)
tarea_sub_2000 = tarea_2000_bins.sel(bins=bin)
tarea_sub_2090 = tarea_2090_bins.sel(bins=bin)

Fraction of total area by region that is EXCEPTIONAL AEV value


In [31]:
# calculate and print regional area weighted averages
print('Antarctic peninsula region')
reg = 'mpa_plan_1_mod'

print('Observational fraction')
avg_pd = tarea_sub_pd.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_pd.values*100)

print('Model 2000s fraction')
avg_2000 = tarea_sub_2000.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2000.values*100)

print('Model 2090s fraction')
avg_2090 = tarea_sub_2090.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2090.values*100)

Antarctic peninsula region
Observational fraction
1.0112218389076415
Model 2000s fraction
0.24489514669275184
Model 2090s fraction
0.29642162943511585


In [32]:
# calculate and print regional area weighted averages
print('Weddell region')
reg = 'mpa_plan_3_4'

print('Observational fraction')
avg_pd = tarea_sub_pd.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_pd.values*100)

print('Model 2000s fraction')
avg_2000 = tarea_sub_2000.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2000.values*100)

print('Model 2090s fraction')
avg_2090 = tarea_sub_2090.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2090.values*100)

Weddell region
Observational fraction
0.2202618937874671
Model 2000s fraction
0.37809663233020324
Model 2090s fraction
0.1290787712651216


In [33]:
# calculate and print regional area weighted averages
print('East Antarctic region')
reg = 'mpa_plan_7_mod'

print('Observational fraction')
avg_pd = tarea_sub_pd.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_pd.values*100)

print('Model 2000s fraction')
avg_2000 = tarea_sub_2000.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2000.values*100)

print('Model 2090s fraction')
avg_2090 = tarea_sub_2090.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2090.values*100)

East Antarctic region
Observational fraction
1.6540345473695668
Model 2000s fraction
0.6100317318760746
Model 2090s fraction
1.3499910146250342


In [34]:
# calculate and print regional area weighted averages
print('Ross region')
reg = 'mpa_plan_8'

print('Observational fraction')
avg_pd = tarea_sub_pd.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_pd.values*100)

print('Model 2000s fraction')
avg_2000 = tarea_sub_2000.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2000.values*100)

print('Model 2090s fraction')
avg_2090 = tarea_sub_2090.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2090.values*100)

Ross region
Observational fraction
2.989249014447649
Model 2000s fraction
5.760547500970175
Model 2090s fraction
5.394704708104734


In [35]:
# calculate and print regional area weighted averages
print('Amundsen region')
reg = 'mpa_plan_9'

print('Observational fraction')
avg_pd = tarea_sub_pd.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_pd.values*100)

print('Model 2000s fraction')
avg_2000 = tarea_sub_2000.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2000.values*100)

print('Model 2090s fraction')
avg_2090 = tarea_sub_2090.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2090.values*100)

Amundsen region
Observational fraction
0.38699454631976515
Model 2000s fraction
0.3399919370151263
Model 2090s fraction
0.24598671840584865


#### High

In [36]:
print('Fraction of total area by region that is HIGH AEV value')
# keep high points
bin = 2

tarea_sub_pd = tarea_pd_bins.sel(bins=bin)
tarea_sub_2000 = tarea_2000_bins.sel(bins=bin)
tarea_sub_2090 = tarea_2090_bins.sel(bins=bin)

Fraction of total area by region that is HIGH AEV value


In [37]:
# calculate and print regional area weighted averages
print('Antarctic peninsula region')
reg = 'mpa_plan_1_mod'

print('Observational fraction')
avg_pd = tarea_sub_pd.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_pd.values*100)

print('Model 2000s fraction')
avg_2000 = tarea_sub_2000.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2000.values*100)

print('Model 2090s fraction')
avg_2090 = tarea_sub_2090.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2090.values*100)

Antarctic peninsula region
Observational fraction
2.76856344936064
Model 2000s fraction
1.466998532808912
Model 2090s fraction
0.7456608904479347


In [38]:
# calculate and print regional area weighted averages
print('Weddell region')
reg = 'mpa_plan_3_4'

print('Observational fraction')
avg_pd = tarea_sub_pd.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_pd.values*100)

print('Model 2000s fraction')
avg_2000 = tarea_sub_2000.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2000.values*100)

print('Model 2090s fraction')
avg_2090 = tarea_sub_2090.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2090.values*100)

Weddell region
Observational fraction
1.712105755619822
Model 2000s fraction
1.2799194301682169
Model 2090s fraction
0.8186050816645951


In [39]:
# calculate and print regional area weighted averages
print('East Antarctic region')
reg = 'mpa_plan_7_mod'

print('Observational fraction')
avg_pd = tarea_sub_pd.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_pd.values*100)

print('Model 2000s fraction')
avg_2000 = tarea_sub_2000.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2000.values*100)

print('Model 2090s fraction')
avg_2090 = tarea_sub_2090.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2090.values*100)

East Antarctic region
Observational fraction
5.479509294084669
Model 2000s fraction
5.628083369148
Model 2090s fraction
8.40044432708673


In [40]:
# calculate and print regional area weighted averages
print('Ross region')
reg = 'mpa_plan_8'

print('Observational fraction')
avg_pd = tarea_sub_pd.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_pd.values*100)

print('Model 2000s fraction')
avg_2000 = tarea_sub_2000.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2000.values*100)

print('Model 2090s fraction')
avg_2090 = tarea_sub_2090.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2090.values*100)

Ross region
Observational fraction
4.669912118802533
Model 2000s fraction
4.974360775710849
Model 2090s fraction
5.981051930937753


In [41]:
# calculate and print regional area weighted averages
print('Amundsen region')
reg = 'mpa_plan_9'

print('Observational fraction')
avg_pd = tarea_sub_pd.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_pd.values*100)

print('Model 2000s fraction')
avg_2000 = tarea_sub_2000.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2000.values*100)

print('Model 2090s fraction')
avg_2090 = tarea_sub_2090.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2090.values*100)

Amundsen region
Observational fraction
0.8930061188276208
Model 2000s fraction
0.34311688031062204
Model 2090s fraction
0.4840879322896876


#### Moderate

In [42]:
print('Fraction of total area by region that is MODERATE AEV value')
# keep moderate points
bin = 3

tarea_sub_pd = tarea_pd_bins.sel(bins=bin)
tarea_sub_2000 = tarea_2000_bins.sel(bins=bin)
tarea_sub_2090 = tarea_2090_bins.sel(bins=bin)

Fraction of total area by region that is MODERATE AEV value


In [43]:
# calculate and print regional area weighted averages
print('Antarctic peninsula region')
reg = 'mpa_plan_1_mod'

print('Observational fraction')
avg_pd = tarea_sub_pd.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_pd.values*100)

print('Model 2000s fraction')
avg_2000 = tarea_sub_2000.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2000.values*100)

print('Model 2090s fraction')
avg_2090 = tarea_sub_2090.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2090.values*100)

Antarctic peninsula region
Observational fraction
9.439046555081902
Model 2000s fraction
6.161707136396976
Model 2090s fraction
3.2085509399888705


In [44]:
# calculate and print regional area weighted averages
print('Weddell region')
reg = 'mpa_plan_3_4'

print('Observational fraction')
avg_pd = tarea_sub_pd.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_pd.values*100)

print('Model 2000s fraction')
avg_2000 = tarea_sub_2000.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2000.values*100)

print('Model 2090s fraction')
avg_2090 = tarea_sub_2090.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2090.values*100)

Weddell region
Observational fraction
20.63986299341773
Model 2000s fraction
25.47863094473043
Model 2090s fraction
44.236035844815106


In [45]:
# calculate and print regional area weighted averages
print('East Antarctic region')
reg = 'mpa_plan_7_mod'

print('Observational fraction')
avg_pd = tarea_sub_pd.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_pd.values*100)

print('Model 2000s fraction')
avg_2000 = tarea_sub_2000.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2000.values*100)

print('Model 2090s fraction')
avg_2090 = tarea_sub_2090.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2090.values*100)

East Antarctic region
Observational fraction
54.86500470571458
Model 2000s fraction
65.0761088967187
Model 2090s fraction
36.65069189220114


In [46]:
# calculate and print regional area weighted averages
print('Ross region')
reg = 'mpa_plan_8'

print('Observational fraction')
avg_pd = tarea_sub_pd.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_pd.values*100)

print('Model 2000s fraction')
avg_2000 = tarea_sub_2000.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2000.values*100)

print('Model 2090s fraction')
avg_2090 = tarea_sub_2090.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2090.values*100)

Ross region
Observational fraction
10.650665562090044
Model 2000s fraction
15.15216317539519
Model 2090s fraction
30.450157956042002


In [47]:
# calculate and print regional area weighted averages
print('Amundsen region')
reg = 'mpa_plan_9'

print('Observational fraction')
avg_pd = tarea_sub_pd.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_pd.values*100)

print('Model 2000s fraction')
avg_2000 = tarea_sub_2000.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2000.values*100)

print('Model 2090s fraction')
avg_2090 = tarea_sub_2090.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2090.values*100)

Amundsen region
Observational fraction
20.24780555675124
Model 2000s fraction
0.979122564480469
Model 2090s fraction
1.8311957661096734


#### Low

In [48]:
print('Fraction of total area by region that is LOW AEV value')
# keep low points
bin = 4

tarea_sub_pd = tarea_pd_bins.sel(bins=bin)
tarea_sub_2000 = tarea_2000_bins.sel(bins=bin)
tarea_sub_2090 = tarea_2090_bins.sel(bins=bin)

Fraction of total area by region that is LOW AEV value


In [49]:
# calculate and print regional area weighted averages
print('Antarctic peninsula region')
reg = 'mpa_plan_1_mod'

print('Observational fraction')
avg_pd = tarea_sub_pd.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_pd.values*100)

print('Model 2000s fraction')
avg_2000 = tarea_sub_2000.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2000.values*100)

print('Model 2090s fraction')
avg_2090 = tarea_sub_2090.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2090.values*100)

Antarctic peninsula region
Observational fraction
86.78116815664981
Model 2000s fraction
92.12639918410136
Model 2090s fraction
95.74936654012808


In [50]:
# calculate and print regional area weighted averages
print('Weddell region')
reg = 'mpa_plan_3_4'

print('Observational fraction')
avg_pd = tarea_sub_pd.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_pd.values*100)

print('Model 2000s fraction')
avg_2000 = tarea_sub_2000.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2000.values*100)

print('Model 2090s fraction')
avg_2090 = tarea_sub_2090.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2090.values*100)

Weddell region
Observational fraction
77.427769357175
Model 2000s fraction
72.86335299277114
Model 2090s fraction
54.81628030225518


In [51]:
# calculate and print regional area weighted averages
print('East Antarctic region')
reg = 'mpa_plan_7_mod'

print('Observational fraction')
avg_pd = tarea_sub_pd.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_pd.values*100)

print('Model 2000s fraction')
avg_2000 = tarea_sub_2000.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2000.values*100)

print('Model 2090s fraction')
avg_2090 = tarea_sub_2090.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2090.values*100)

East Antarctic region
Observational fraction
38.00145145283119
Model 2000s fraction
28.685776002257253
Model 2090s fraction
53.598872766087105


In [52]:
# calculate and print regional area weighted averages
print('Ross region')
reg = 'mpa_plan_8'

print('Observational fraction')
avg_pd = tarea_sub_pd.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_pd.values*100)

print('Model 2000s fraction')
avg_2000 = tarea_sub_2000.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2000.values*100)

print('Model 2090s fraction')
avg_2090 = tarea_sub_2090.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2090.values*100)

Ross region
Observational fraction
81.69017330465978
Model 2000s fraction
74.1129285479238
Model 2090s fraction
58.17408540491553


In [53]:
# calculate and print regional area weighted averages
print('Amundsen region')
reg = 'mpa_plan_9'

print('Observational fraction')
avg_pd = tarea_sub_pd.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_pd.values*100)

print('Model 2000s fraction')
avg_2000 = tarea_sub_2000.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2000.values*100)

print('Model 2090s fraction')
avg_2090 = tarea_sub_2090.sel(region=reg).sum(dim=['nlat','nlon'])/tarea_region.sel(region=reg).sum(dim=['nlat','nlon'])
print(avg_2090.values*100)

Amundsen region
Observational fraction
78.47219377810138
Model 2000s fraction
98.33776861819379
Model 2090s fraction
97.4387295831948
